# ARGOS - Gerador Automático de Data Schemas

Este notebook gera automaticamente os arquivos de schema para todas as tabelas do projeto ARGOS.

## O que será gerado:

Para cada tabela:
- `DESCRIBE FORMATTED` - estrutura completa da tabela
- `SELECT * FROM ... LIMIT 10` - amostra de dados

## Tabelas processadas:

### Originais (4 tabelas)
- nfce.nfce
- niat.argos_cnpj
- niat.tabela_niat
- usr_sat_ods.vw_ods_contrib

### Intermediárias (4 tabelas)
- niat.argos_nfce_base_extraida
- niat.argos_nfce_periodo_base
- niat.argos_medias_historicas_produto
- niat.argos_mudanca_comportamento

### Views (1 tabela)
- niat.argos_vw_evolucao_nfce ⭐ (mais importante - usada em todo o Streamlit)

---

## 1. Importar Bibliotecas

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import pandas as pd
from datetime import datetime
import os

print("✅ Bibliotecas importadas com sucesso")
print(f"📅 Data/Hora: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## 2. Configurar Lista de Tabelas

In [ ]:
# Lista completa de tabelas a serem documentadas
TABELAS = {
    'ORIGINAIS': [
        'nfce.nfce',
        'niat.argos_cnpj',
        'niat.tabela_niat',
        'usr_sat_ods.vw_ods_contrib'
    ],
    'INTERMEDIARIAS': [
        'niat.argos_nfce_base_extraida',
        'niat.argos_nfce_periodo_base',
        'niat.argos_medias_historicas_produto',
        'niat.argos_mudanca_comportamento'
    ],
    'VIEWS': [
        'niat.argos_vw_evolucao_nfce'
    ]
}

# Diretório de saída
OUTPUT_DIR = './data-schemas'

# Contagem total
total = sum(len(v) for v in TABELAS.values())
print(f"📊 Total de tabelas configuradas: {total}")
for categoria, lista in TABELAS.items():
    print(f"   - {categoria}: {len(lista)} tabelas")

## 3. Criar Diretório de Saída

In [ ]:
# Criar diretório se não existir
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)
    print(f"✅ Diretório criado: {OUTPUT_DIR}")
else:
    print(f"ℹ️  Diretório já existe: {OUTPUT_DIR}")

## 4. Funções Auxiliares

In [ ]:
def obter_nome_arquivo(tabela_completa):
    """Converte nome da tabela em nome de arquivo"""
    nome = tabela_completa.split('.')[-1]
    return f"{nome}_schema.txt"

def formatar_header(titulo, char='='):
    """Formata header para os arquivos"""
    linha = char * 80
    return f"\n{linha}\n{titulo.center(80)}\n{linha}\n"

def salvar_schema(tabela, describe_df, sample_df, categoria):
    """Salva schema completo em arquivo"""
    nome_arquivo = obter_nome_arquivo(tabela)
    caminho_completo = os.path.join(OUTPUT_DIR, nome_arquivo)

    with open(caminho_completo, 'w', encoding='utf-8') as f:
        # Header do arquivo
        f.write(formatar_header(f'DATA SCHEMA - {tabela}'))
        f.write(f"\nCategoria: {categoria}\n")
        f.write(f"Gerado em: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write(f"Tabela: {tabela}\n")

        # DESCRIBE FORMATTED
        f.write(formatar_header('DESCRIBE FORMATTED', '-'))
        f.write("\n```sql\n")
        f.write(f"DESCRIBE FORMATTED {tabela};\n")
        f.write("```\n\n")
        
        describe_str = describe_df.toPandas().to_string(index=False, max_rows=None)
        f.write(describe_str)
        f.write("\n")

        # SELECT LIMIT 10
        f.write(formatar_header('SELECT * FROM ... LIMIT 10', '-'))
        f.write("\n```sql\n")
        f.write(f"SELECT * FROM {tabela} LIMIT 10;\n")
        f.write("```\n\n")
        
        sample_str = sample_df.toPandas().to_string(index=False, max_rows=None)
        f.write(sample_str)
        f.write("\n")

        # Footer
        f.write(formatar_header('FIM DO SCHEMA', '='))

    return caminho_completo

print("✅ Funções auxiliares definidas")

## 5. Função de Extração de Schema

In [ ]:
def extrair_schema_tabela(tabela, categoria):
    """
    Extrai schema e dados de exemplo de uma tabela
    
    Args:
        tabela: Nome completo da tabela (schema.tabela)
        categoria: Categoria da tabela
        
    Returns:
        True se sucesso, False se erro
    """
    print(f"\n{'='*80}")
    print(f"Processando: {tabela} ({categoria})")
    print(f"{'='*80}")

    try:
        # 1. DESCRIBE FORMATTED
        print(f"⏳ Executando DESCRIBE FORMATTED {tabela}...")
        describe_query = f"DESCRIBE FORMATTED {tabela}"
        describe_df = spark.sql(describe_query)
        num_linhas = describe_df.count()
        print(f"✅ DESCRIBE obtido - {num_linhas} linhas")

        # 2. SELECT LIMIT 10
        print(f"⏳ Executando SELECT * FROM {tabela} LIMIT 10...")
        sample_query = f"SELECT * FROM {tabela} LIMIT 10"
        sample_df = spark.sql(sample_query)
        num_colunas = len(sample_df.columns)
        num_registros = sample_df.count()
        print(f"✅ Sample obtido - {num_colunas} colunas, {num_registros} registros")

        # 3. Salvar em arquivo
        print(f"💾 Salvando schema em arquivo...")
        caminho = salvar_schema(tabela, describe_df, sample_df, categoria)
        print(f"✅ Schema salvo: {caminho}")
        
        # 4. Exibir preview
        print(f"\n📋 Preview da estrutura:")
        sample_df.printSchema()
        
        print(f"\n📊 Preview dos dados (primeiras 3 linhas):")
        sample_df.show(3, truncate=True, vertical=False)

        return True

    except Exception as e:
        print(f"❌ ERRO ao processar {tabela}: {str(e)}")
        
        # Salvar erro em arquivo
        nome_arquivo = obter_nome_arquivo(tabela)
        caminho_erro = os.path.join(OUTPUT_DIR, f"ERRO_{nome_arquivo}")
        
        with open(caminho_erro, 'w', encoding='utf-8') as f:
            f.write(formatar_header(f'ERRO - {tabela}'))
            f.write(f"\nCategoria: {categoria}\n")
            f.write(f"Erro em: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
            f.write(f"\nMensagem de erro:\n{str(e)}\n")
        
        print(f"⚠️  Erro salvo em: {caminho_erro}")
        return False

print("✅ Função de extração definida")

---

## 6. 🚀 EXECUTAR GERAÇÃO DE SCHEMAS

⚠️ **ATENÇÃO**: Execute a célula abaixo para processar TODAS as tabelas de uma vez.

Ou execute as células seguintes para processar por categoria.

In [ ]:
# OPÇÃO 1: EXECUTAR TODAS AS TABELAS DE UMA VEZ

print("\n" + "="*80)
print("ARGOS - GERADOR AUTOMÁTICO DE DATA SCHEMAS".center(80))
print("="*80)

# Contadores
total_tabelas = sum(len(tabelas) for tabelas in TABELAS.values())
processadas = 0
sucesso = 0
falhas = 0

# Processar cada categoria
for categoria, lista_tabelas in TABELAS.items():
    print(f"\n{'#'*80}")
    print(f"CATEGORIA: {categoria}".center(80))
    print(f"{'#'*80}")
    
    for tabela in lista_tabelas:
        processadas += 1
        resultado = extrair_schema_tabela(tabela, categoria)
        
        if resultado:
            sucesso += 1
        else:
            falhas += 1
        
        print(f"\n📊 Progresso: {processadas}/{total_tabelas} tabelas processadas")

# Resumo final
print("\n" + "="*80)
print("RESUMO FINAL".center(80))
print("="*80)
print(f"✅ Total de tabelas: {total_tabelas}")
print(f"✅ Processadas com sucesso: {sucesso}")
print(f"❌ Falhas: {falhas}")
print(f"📁 Arquivos salvos em: {OUTPUT_DIR}")
print("="*80 + "\n")

---

## 7. OPÇÃO 2: Executar por Categoria (Opcional)

Se preferir processar uma categoria por vez, execute as células abaixo individualmente.

In [ ]:
# Processar apenas TABELAS ORIGINAIS
print("\n" + "#"*80)
print("PROCESSANDO: TABELAS ORIGINAIS".center(80))
print("#"*80)

for tabela in TABELAS['ORIGINAIS']:
    extrair_schema_tabela(tabela, 'ORIGINAIS')

In [ ]:
# Processar apenas TABELAS INTERMEDIÁRIAS
print("\n" + "#"*80)
print("PROCESSANDO: TABELAS INTERMEDIÁRIAS".center(80))
print("#"*80)

for tabela in TABELAS['INTERMEDIARIAS']:
    extrair_schema_tabela(tabela, 'INTERMEDIARIAS')

In [ ]:
# Processar apenas VIEWS
print("\n" + "#"*80)
print("PROCESSANDO: VIEWS".center(80))
print("#"*80)

for tabela in TABELAS['VIEWS']:
    extrair_schema_tabela(tabela, 'VIEWS')

---

## 8. OPÇÃO 3: Processar Tabela Individual (Opcional)

Para processar uma tabela específica:

In [ ]:
# Processar apenas uma tabela específica
# Altere o nome da tabela conforme necessário

tabela_especifica = 'niat.argos_vw_evolucao_nfce'
categoria_especifica = 'VIEWS'

extrair_schema_tabela(tabela_especifica, categoria_especifica)

---

## 9. Gerar Índice README

In [ ]:
# Gerar arquivo README.md com índice de todos os schemas

indice_path = os.path.join(OUTPUT_DIR, 'README.md')

with open(indice_path, 'w', encoding='utf-8') as f:
    f.write("# ARGOS - Data Schemas\n\n")
    f.write(f"Gerado em: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
    f.write("---\n\n")

    f.write("## Estrutura de Dados\n\n")
    f.write("Este diretório contém os schemas completos de todas as tabelas utilizadas no projeto ARGOS.\n\n")

    # Listar por categoria
    for categoria, lista_tabelas in TABELAS.items():
        f.write(f"### {categoria}\n\n")
        
        for tabela in lista_tabelas:
            nome_arquivo = obter_nome_arquivo(tabela)
            f.write(f"- **{tabela}** → [`{nome_arquivo}`](./{nome_arquivo})\n")
        
        f.write("\n")

    # Informações adicionais
    f.write("---\n\n")
    f.write("## Formato dos Arquivos\n\n")
    f.write("Cada arquivo contém:\n\n")
    f.write("1. **DESCRIBE FORMATTED** - Estrutura completa da tabela\n")
    f.write("2. **SELECT LIMIT 10** - Amostra de 10 registros\n\n")

    f.write("## Como Regenerar\n\n")
    f.write("Para regenerar todos os schemas, execute o notebook:\n\n")
    f.write("`Gerar_Data_Schemas.ipynb`\n")

print(f"✅ Índice gerado: {indice_path}")
print(f"📁 Arquivos disponíveis em: {OUTPUT_DIR}")

---

## 10. Listar Arquivos Gerados

In [ ]:
# Listar todos os arquivos gerados no diretório de saída

import os

arquivos = sorted(os.listdir(OUTPUT_DIR))

print(f"\n📁 Arquivos em {OUTPUT_DIR}:")
print("="*80)

schemas = [f for f in arquivos if f.endswith('_schema.txt')]
erros = [f for f in arquivos if f.startswith('ERRO_')]
outros = [f for f in arquivos if f not in schemas and f not in erros]

print(f"\n✅ Schemas gerados ({len(schemas)}):")
for arquivo in schemas:
    tamanho = os.path.getsize(os.path.join(OUTPUT_DIR, arquivo))
    print(f"   - {arquivo} ({tamanho:,} bytes)")

if erros:
    print(f"\n❌ Arquivos de erro ({len(erros)}):")
    for arquivo in erros:
        print(f"   - {arquivo}")

if outros:
    print(f"\nℹ️  Outros arquivos ({len(outros)}):")
    for arquivo in outros:
        print(f"   - {arquivo}")

print("\n" + "="*80)

---

## ✅ Concluído!

Os schemas foram gerados com sucesso no diretório `./data-schemas/`.

### Próximos passos:

1. Revise os arquivos gerados em `./data-schemas/`
2. Verifique se há arquivos de erro e investigue as causas
3. Use os schemas para documentação do projeto

### Estrutura gerada:

```
data-schemas/
├── README.md                                    # Índice de todos os schemas
├── nfce_schema.txt                             # Schema da tabela nfce.nfce
├── argos_cnpj_schema.txt                       # Schema da tabela niat.argos_cnpj
├── tabela_niat_schema.txt                      # Schema da tabela niat.tabela_niat
├── vw_ods_contrib_schema.txt                   # Schema da view usr_sat_ods.vw_ods_contrib
├── argos_nfce_base_extraida_schema.txt         # Schema da tabela intermediária
├── argos_nfce_periodo_base_schema.txt          # Schema da tabela intermediária
├── argos_medias_historicas_produto_schema.txt  # Schema da tabela intermediária
├── argos_mudanca_comportamento_schema.txt      # Schema da tabela intermediária
└── argos_vw_evolucao_nfce_schema.txt          # Schema da view principal ⭐
```